# **Procesamiento de Lenguaje Natural**

## **Tarea 2 - Notebook 1**

## Integrantes

* ### Daniel Osorio Cárdenas
* ### Juan Diego Calixto Núñez

Este Notebook incluye los literales I, II, III, IV de la tarea. Estos corresponden a la construcción de los modelos de lenguaje que se usarán en el Notebook 2.